In [1]:
#region Import the necessary libraries
import pandas as pd
from datetime import datetime, timedelta
#end region

def read_data(file_path, sheet_name):
    """Reads data from an Excel file and returns a DataFrame."""
    try:
        return pd.read_excel(file_path, sheet_name=sheet_name)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return None

def clean_data(df_joining_prediction):
    """Trims leading and trailing spaces and standardizes case of column values."""
    try:
        df_joining_prediction = df_joining_prediction.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        df_joining_prediction = df_joining_prediction.apply(
            lambda col: col.str.title() if pd.api.types.is_string_dtype(col) else col)
        return df_joining_prediction
    except Exception as e:
        print(f"Error cleaning data: {e}")
        return None

today = datetime.today()
def get_future_dates():
    """Calculates dates after 7, 14, 30, 45, 60 days and 2 months."""
    try:
        dates = [
            today + timedelta(days=7),
            today + timedelta(days=14),
            today + timedelta(days=30),
            today + timedelta(days=45),
            today + timedelta(days=60),
            today + timedelta(days=240)
        ]
        return dates
    except Exception as e:
        print(f"Error calculating future dates: {e}")
        return None

def filter_and_aggregate(df_joining_prediction, column, dates):
    """Filters and aggregates data based on DOJ and specified column."""
    try:
        results = []
        for i, date in enumerate(dates):
            if i == 0:
                filtered_df = df_joining_prediction[
                    (df_joining_prediction['DOJ'] >= today) &
                    (df_joining_prediction['DOJ'] <= date)
                ]
            else:
                filtered_df = df_joining_prediction[
                    (df_joining_prediction['DOJ'] >= dates[i-1]) &
                    (df_joining_prediction['DOJ'] <= date)
                ]

            final_result_df = filtered_df.groupby(column).size().reset_index(name=f'After {i+1} Months')
            results.append(final_result_df)

        merged_df = results[0]
        for i in range(1, len(results)):
            merged_df = merged_df.merge(results[i], on=column, how='outer')

        merged_df = merged_df.fillna(0)
        merged_df = merged_df.sort_values(by=column)
        column_names = [column] + [f'After {i+1} Months' for i in range(len(dates))]
        merged_df.columns = column_names
        merged_df['Total'] = merged_df[column_names[1:]].sum(axis=1)

        return merged_df
    except Exception as e:
        print(f"Error filtering and aggregating data: {e}")
        return None

def add_grand_total(merged_df, column):
    """Adds a Grand Total row to the DataFrame."""
    try:
        total_row = merged_df.sum(numeric_only=True)
        total_row.name = 'Grand Total'
        total_row = pd.DataFrame(total_row).T
        merged_df_new = pd.concat([merged_df, total_row], ignore_index=True)
        merged_df_new.at[len(merged_df_new) - 1, column] = 'Grand Total'
        return merged_df_new
    except Exception as e:
        print(f"Error adding Grand Total: {e}")
        return None

def sort_and_output(merged_df_new, column):
    """Sorts the DataFrame and returns the final DataFrame."""
    try:
        grand_total_row = merged_df_new[merged_df_new[column] == 'Grand Total']
        without_grand_total_row = merged_df_new[merged_df_new[column] != 'Grand Total']
        merged_df_sorted = without_grand_total_row.sort_values(by='Total', ascending=False)
        df_final_new = pd.concat([merged_df_sorted, grand_total_row])
        df_final_new.reset_index(drop=True, inplace=True)
        return df_final_new
    except Exception as e:
        print(f"Error sorting and outputting data: {e}")
        return None

def write_to_excel(dataframes, excel_file):
    """Writes multiple DataFrames to a single Excel sheet."""
    try:
        start_row = 0
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            for df_joining_prediction in dataframes:
                df_joining_prediction.to_excel(writer, sheet_name='Sheet1', startrow=start_row, index=False)
                start_row += len(df_joining_prediction) + 2
    except Exception as e:
        print(f"Error writing to Excel: {e}")

# Load data
file_path = "Projects/DEMO.xlsx"
sheet_name = "Base"
df_joining_prediction = read_data(file_path, sheet_name)

if df_joining_prediction is None:
    exit()

df_joining_prediction = clean_data(df_joining_prediction)
if df_joining_prediction is None:
    exit()

dates = get_future_dates()
if dates is None:
    exit()

columns = ['Joining location', 'Offered Grade', 'Practice', 'Skill']
dataframes = []
for column in columns:
    merged_df = filter_and_aggregate(df_joining_prediction, column, dates)
    if merged_df is None:
        continue
    merged_df_new = add_grand_total(merged_df, column)
    if merged_df_new is None:
        continue
    df_final_new = sort_and_output(merged_df_new, column)
    if df_final_new is None:
        continue
    dataframes.append(df_final_new)

excel_file = "Projects/DEMO.xlsx"
if dataframes:
    write_to_excel(dataframes, excel_file)
    print(f"Output saved to: {excel_file}")
else:
    print("No data to write. Please check input file or data filters.")


Error reading Excel file: No such keys(s): 'io.excel.zip.reader'
Error cleaning data: 'NoneType' object has no attribute 'applymap'
Error filtering and aggregating data: 'NoneType' object is not subscriptable
Error filtering and aggregating data: 'NoneType' object is not subscriptable
Error filtering and aggregating data: 'NoneType' object is not subscriptable
Error filtering and aggregating data: 'NoneType' object is not subscriptable
⚠️ No data to write. Please check input file or data filters.
